In [1]:
import requests
import json
import pandas as pd

In [2]:
token = '61b0de5e-5006-0203-de71-d778b72cc02d'

In [3]:
def url_to_df(url: str, token: str, main_col_name: str):
    """This function receives the INEGI API url, and the token,
    makes the petition and if the response is 200, then json_to_df
    is called and the response json is passed. The df is returned."""
    res = requests.get(url.replace('[Aquí va tu Token]', token))
    if res.status_code == 200:
        df = json_to_df(res.json(), main_col_name)
        return df
    return None

def json_to_df(response_json, main_col_name):
    df = pd.json_normalize(response_json['Series'][0]['OBSERVATIONS'])
    df = df[['TIME_PERIOD', 'OBS_VALUE']].rename(columns={'OBS_VALUE': main_col_name, 'TIME_PERIOD': 'year'}, inplace=False)
    return df

In [4]:
nacimientos_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000026/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
defunciones_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000030/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
matrimonios_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000038/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
divorcios_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000039/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
poblacion_total_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000001/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
homicidios_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/6300000264/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
porcentaje_pib_educacion_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/6207067825/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
suicidios_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/6200240338/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'
median_age_url = 'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000010/es/0700/false/BISE/2.0/[Aquí va tu Token]?type=json'

In [5]:
nacimientos = url_to_df(nacimientos_url, token, 'nacimientos')
defunciones = url_to_df(defunciones_url, token, 'defunciones')
matrimonios = url_to_df(matrimonios_url, token, 'matrimonios')
divorcios = url_to_df(divorcios_url, token, 'divorcios')
poblacion_total = url_to_df(poblacion_total_url, token, 'poblacion_total')
porcentaje_pib_educacion = url_to_df(porcentaje_pib_educacion_url, token, 'porcentaje_pib_educacion')
homicidios_total = url_to_df(homicidios_url, token, 'homicidios_total')
suicidios = url_to_df(suicidios_url, token, 'suicidios')
median_age = url_to_df(median_age_url, token, 'median_age')

In [6]:
(
    pd.merge(right=nacimientos ,left=defunciones ,how='inner', left_on='year', right_on='year')
    .merge(right=matrimonios, left_on='year',right_on='year')
    .merge(right=divorcios, left_on='year',right_on='year')
    .merge(right=poblacion_total, left_on='year',right_on='year', how='left')
    .merge(right=porcentaje_pib_educacion, left_on='year',right_on='year', how='left')
    .merge(right=homicidios_total, left_on='year',right_on='year', how='left')
    .merge(right=suicidios, left_on='year',right_on='year', how='left')
    .merge(right=median_age, left_on='year',right_on='year', how='left')
)

,year,defunciones,nacimientos,matrimonios,divorcios,poblacion_total,porcentaje_pib_educacion,homicidios_total,suicidios,median_age
0,1994,419074.00000000000000000000,2904389.00000000000000000000,671640.00000000000000000000,35029.00000000000000000000,NaN,4.34143060992847300000,15839.00000000000000000000,2603.00000000000000000000,NaN
1,1995,430278.00000000000000000000,2750444.00000000000000000000,658114.00000000000000000000,37455.00000000000000000000,91158290.00000000000000000000,3.89854368974041500000,15612.00000000000000000000,2894.00000000000000000000,21.00000000000000000000
2,1996,436321.00000000000000000000,2707718.00000000000000000000,670523.00000000000000000000,38545.00000000000000000000,NaN,4.76065648725909100000,14505.00000000000000000000,3020.00000000000000000000,NaN
3,1997,440437.00000000000000000000,2698425.00000000000000000000,707840.00000000000000000000,40792.00000000000000000000,NaN,4.74841035663127800000,13552.00000000000000000000,3370.00000000000000000000,NaN
4,1998,444665.00000000000000000000,2668428.00000000000000000000,704456.00000000000000000000,45889.00000000000000000000,NaN,5.12610592286309500000,13656.00000000000000000000,3342.00000000000000000000,NaN
5,1999,443950.00000000000000000000,2769089.00000000000000000000,743856.00000000000000000000,49271.00000000000000000000,NaN,5.23157756794237400000,12249.00000000000000000000,3339.00000000000000000000,NaN
6,2000,437667.00000000000000000000,2798339.00000000000000000000,707422.00000000000000000000,52358.00000000000000000000,97483412.00000000000000000000,5.27441171026473700000,10737.00000000000000000000,3475.00000000000000000000,22.00000000000000000000
7,2001,443127.00000000000000000000,2767610.00000000000000000000,665434.00000000000000000000,57370.00000000000000000000,NaN,5.58303511044891200000,10285.00000000000000000000,3811.00000000000000000000,NaN
8,2002,459687.00000000000000000000,2699084.00000000000000000000,616654.00000000000000000000,60641.00000000000000000000,NaN,5.89349897839958600000,10088.00000000000000000000,3871.00000000000000000000,NaN
9,2003,472140.00000000000000000000,2655894.00000000000000000000,584142.00000000000000000000,64248.00000000000000000000,NaN,6.29206322175780000000,10087.00000000000000000000,4104.00000000000000000000,NaN


### Crecimiento poblacional en México

Cambio de población total
Población por estado
Edad media de población


Set de datas del banco mundial

In [7]:
pre = pd.read_csv('datasets/API_MEX_DS2_en_csv_v2_32388.csv', skiprows=[0,1,2,3]).drop(['Country Code', 'Country Name', '2023', 'Unnamed: 68'], axis=1)

In [8]:
pre.head()

,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,"Intentional homicides, female (per 100,000 fem...",VC.IHR.PSRC.FE.P5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.423447,3.972907,3.885846,4.534932,5.469392,5.924644,6.092515,6.143375,6.171259,NaN
1,Battle-related deaths (number of people),VC.BTL.DETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Voice and Accountability: Percentile Rank,VA.PER.RNK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,53.051643,48.768475,44.334976,43.842365,42.857143,45.631069,45.893719,44.927536,43.961353,42.028984
3,Transport services (% of commercial service ex...,TX.VAL.TRAN.ZS.WT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.743973,16.970192,16.621786,15.260647,14.625960,15.362666,14.250811,18.474154,14.094799,12.823647
4,"Computer, communications and other services (%...",TX.VAL.OTHR.ZS.WT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.111100,18.012572,18.038976,18.215565,17.815778,17.739717,17.381470,25.280193,22.236580,19.046960


set de datos INEGI

In [10]:
pd.read_excel('datasets/estructura_00.xlsx')

,cve_entidad,desc_entidad,cve_municipio,desc_municipio,id_indicador,indicador,1900,1910,1921,1930,...,2016,2017,2018,2019,2020,2021,2030,2040,2050,unidad_medida
0,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000001,Población total,NaN,15160369.0,14334780.0,16552722.0,...,NaN,NaN,NaN,NaN,1.260140e+08,NaN,NaN,NaN,NaN,Número de personas
1,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000002,Población total hombres,NaN,7504471.0,7003785.0,8119004.0,...,NaN,NaN,NaN,NaN,6.147339e+07,NaN,NaN,NaN,NaN,Número de personas
2,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000003,Población total mujeres,NaN,7655898.0,7330995.0,8433718.0,...,NaN,NaN,NaN,NaN,6.454063e+07,NaN,NaN,NaN,NaN,Número de personas
3,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000004,Porcentaje de población de 15 a 29 años,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.483031e+01,NaN,NaN,NaN,NaN,No especificada
4,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000005,Porcentaje de hombres de 15 a 29 años,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.525293e+01,NaN,NaN,NaN,NaN,No especificada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332765,32,Zacatecas,58,Santa María de la Paz,6207020032,Porcentaje de hombres,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.878930e+01,NaN,NaN,NaN,NaN,Porcentaje
332766,32,Zacatecas,58,Santa María de la Paz,6207020033,Porcentaje de mujeres,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.121070e+01,NaN,NaN,NaN,NaN,Porcentaje
332767,32,Zacatecas,58,Santa María de la Paz,6207020632,Población total en viviendas particulares habi...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Número de personas
332768,32,Zacatecas,58,Santa María de la Paz,6207129629,Porcentaje de la población nacional/estatal qu...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.705773e-01,NaN,NaN,NaN,NaN,Porcentaje
